# Train test split KVLCC2 HSVA feature importance

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,7)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.prime_system import PrimeSystem

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline
from src.parameters import df_parameters
p = df_parameters['symbol']
from src.feature_selection import drop_multicollinearity

In [ ]:
ship="kvlcc2_hsva"
#vmm_name = "vmm_martins_simple"
vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

regression = catalog.load(f"{ship}.updated.{vmm_name}.joined.regression")
regression.diff_eq_Y.exclude_parameters.pop('Ydelta')
regression.diff_eq_Y.exclude_parameters.pop('Ythrustdelta')


data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")

added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")
data_with_force = predict_force(data=data, added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)

In [ ]:
from src.bias_variance_tradeoff import train_test_split_exteme, train_test_split_run
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
ids = list(data_with_force['id'].unique())
ids_train = ids.copy()
id_test = 'HSVA_CPMC_KVLCC2_Z_35_05'
ids_train.remove(id_test)
mask = data_with_force['id'].isin(ids_train)
data_train = data_with_force.loc[mask].copy()

In [ ]:
ps = PrimeSystem(**ship_data)
#data['U'] = np.sqrt(data['u']**2 + data['v']**2)
data_prime = ps.prime(data_train, U=data_with_force['U'])

In [ ]:
Xs = {}
ys = {}

initial_removes = ['Xuuu','Xuvv']

Xs['X'], ys['X'] = regression.diff_eq_X.calculate_features_and_label(data=data_prime, y=data_prime['fx'])
Xs['X'].drop(columns=initial_removes, inplace=True)

Xs['Y'], ys['Y'] = regression.diff_eq_Y.calculate_features_and_label(data=data_prime, y=data_prime['fy'])
Xs['N'], ys['N'] = regression.diff_eq_N.calculate_features_and_label(data=data_prime, y=data_prime['mz'])

dofs = list(Xs.keys())

In [ ]:
import seaborn as sns
corr = Xs['Y'].corr()
corr = np.tril(corr)
corr = pd.DataFrame(corr, index=Xs['Y'].columns, columns=Xs['Y'].columns)

fig,ax=plt.subplots()
grid = sns.heatmap(corr, cmap='gray_r')

In [ ]:
corr = Xs['Y'].corr().abs()
corr_ = np.tril(corr, k=-1)
corr_ = pd.DataFrame(corr_, index=Xs['Y'].columns, columns=Xs['Y'].columns)

fig,ax=plt.subplots()
grid = sns.heatmap(corr_, cmap='gray_r')

In [ ]:
key = 'Y0u'
buddy = corr_.loc[key].idxmax()

In [ ]:
corr[key].sum()

In [ ]:
corr[key].drop(index=[key,buddy]).max()

In [ ]:
corr[buddy].sum()

In [ ]:
corr[buddy].drop(index=[key,buddy]).max()

In [ ]:
keys = corr_.max().sort_values(ascending=False).index
for key in keys:
    
    buddy = corr_.loc[key].idxmax()
    if buddy!=key:
        #print(f'{key}: {corr[key].sum()}, {buddy}: {corr[buddy].sum()}')
        
        other_corr = corr[key].drop(index=[key,buddy])
        other_corr_budy = corr[buddy].drop(index=[key,buddy])
                
        print(f"{key}:{other_corr.max()}, {buddy}:{other_corr_budy.max()}")
        
    
    

In [ ]:
corr = Xs['Y'].corr().abs()
corr_ = np.tril(corr, k=-1)
corr_ = pd.DataFrame(corr_, index=Xs['Y'].columns, columns=Xs['Y'].columns)
keys = corr_.max().sort_values(ascending=False).index

limit_corr = 0.9

keep_all = False

i = 0
while not keep_all:
    
    print(f'-------- iteration {i} -----------')
    i+=1
    keep_all = True
    
    for key in keys:
        
        if not key in corr:
            continue
        
        buddy = corr_.loc[key].idxmax()
        if buddy!=key:
            #print(f'{key}: {corr[key].sum()}, {buddy}: {corr[buddy].sum()}')
            
            other_corr = corr[key].drop(index=[key,buddy])
            other_corr_budy = corr[buddy].drop(index=[key,buddy])
            
            if np.max([other_corr.max(), other_corr_budy.max()]) > limit_corr:
            
                keep_all = False
                
                if other_corr.max() > other_corr_budy.max():
                    drop = key
                else:
                    drop = buddy
                
                corr.drop(columns=[drop], inplace=True)
                corr.drop(index=[drop], inplace=True)
                corr_.drop(columns=[drop], inplace=True)
                corr_.drop(index=[drop], inplace=True)
                           
                print(f"{key}:{other_corr.max()}, {buddy}:{other_corr_budy.max()} --> drop:{drop}")
            else:
                print(f"{key}:{other_corr.max()}, {buddy}:{other_corr_budy.max()} --> Keep both") 

In [ ]:
fig,ax=plt.subplots()
grid = sns.heatmap(corr_, cmap='gray_r')

In [ ]:
corr_.max().sort_values(ascending=False).plot.bar()

## All dofs

In [ ]:
dofs = list(Xs.keys())

eqs = {

    'X' : regression.X_eq,
    'Y' : regression.Y_eq,
    'N' : regression.N_eq,
    
}

for dof in dofs:
    
    X=Xs[dof]
    X_droped = drop_multicollinearity(X=X, limit_corr=0.90)
    
    fig,axes=plt.subplots(ncols=2)
    for ax,X_ in zip(axes,[X, X_droped]):
        corr = X_.corr().abs()
        corr_ = np.tril(corr, k=-1)
        corr_ = pd.DataFrame(corr_, index=X_.columns, columns=X_.columns)
        corr_.max().sort_values(ascending=False).plot.bar(ax=ax)
        ax.set_ylim(0,1)
    
    removes = list(set(X.columns) - set(X_droped.columns))
    subs = [(p[remove],0) for remove in removes]
    eq = eqs[dof]
    display(eq)
    eq_simplified = eq.subs(subs)
    display(eq_simplified)
    
    
    
    